# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hovd,48.0056,91.6419,-19.15,78,48,1.68,MN,1738282025
1,1,thunder bay,48.4001,-89.3168,-3.01,86,20,2.57,CA,1738281800
2,2,kamuthi,9.4073,78.3734,23.79,87,44,3.65,IN,1738282027
3,3,bilibino,68.0546,166.4372,-39.42,100,89,0.23,RU,1738282028
4,4,hjo,58.3013,14.2878,1.99,96,100,1.76,SE,1738282030


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng", "Lat", 
    geo=True, 
    tiles="OSM", 
    size="Humidity", 
    color="Humidity", 
    cmap="coolwarm", 
    alpha=0.75, 
    frame_width=700, 
    frame_height=500
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Filter cities based on ideal vacation conditions
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with missing values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
64,64,primero de mayo,-32.2552,-58.4228,26.01,32,0,3.59,AR,1738282109
67,67,monte patria,-30.6919,-70.9467,23.29,53,0,2.19,CL,1738282115
261,261,cabo san lucas,22.8909,-109.9124,22.09,49,0,3.09,MX,1738282368
299,299,freetown,8.4840,-13.2299,24.72,82,0,1.90,SL,1738282431
435,435,acajutla,13.5928,-89.8275,26.81,71,0,1.87,SV,1738282606


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Create a copy of the ideal cities DataFrame
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column for "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
64,primero de mayo,AR,-32.2552,-58.4228,32,
67,monte patria,CL,-30.6919,-70.9467,53,
261,cabo san lucas,MX,22.8909,-109.9124,49,
299,freetown,SL,8.4840,-13.2299,82,
435,acajutla,SV,13.5928,-89.8275,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
# Set search radius (in meters)
radius = 10000  

# Base URL for Geoapify Places API
base_url = "https://api.geoapify.com/v2/places"

# Start hotel search
print("Starting hotel search...")

# Iterate through each city in the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # Set parameters for the API request
    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{lng},{lat},{radius}",
        "bias": f"proximity:{lng},{lat}",
        "limit": 1,
        "apiKey": geoapify_key
    }

    # Make the API request
    response = requests.get(base_url, params=params)
    
    # Convert response to JSON format
    name_address = response.json()

    # Try to extract hotel name from response
    try:
        hotel_name = name_address["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log search results
    print(f"{hotel_df.loc[index, 'City']} - Nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display updated hotel_df
hotel_df.head()

Starting hotel search...
primero de mayo - Nearest hotel: No hotel found
monte patria - Nearest hotel: Hotel Monte Cordillera
cabo san lucas - Nearest hotel: Comfort Rooms
freetown - Nearest hotel: Formal Leona hotel
acajutla - Nearest hotel: Hotel Lara
lazaro cardenas - Nearest hotel: Hotel Sol del Pacífico
port augusta - Nearest hotel: Hotel Commonwealth
veracruz - Nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
64,primero de mayo,AR,-32.2552,-58.4228,32,No hotel found
67,monte patria,CL,-30.6919,-70.9467,53,Hotel Monte Cordillera
261,cabo san lucas,MX,22.8909,-109.9124,49,Comfort Rooms
299,freetown,SL,8.4840,-13.2299,82,Formal Leona hotel
435,acajutla,SV,13.5928,-89.8275,71,Hotel Lara


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    tiles="OSM",
    size=100,  # Adjust marker size
    color="Hotel Name",
    hover_cols=["Hotel Name", "City", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Country)

In [30]:
%%capture --no-display

hotel_map = hotel_df.hvplot.points('Lng','Lat',geo=True,tiles='OSM',size='Humidity',color='City',hover_cols = ["Hotel Name", "Country"])

hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)